In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import fire
import json
import os
import numpy as np
import tensorflow as tf

In [10]:
import src.model as model
import src.sample as sample
import src.encoder as encoder


In [24]:
def interact_model(
    model_name='1558M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    top_p=1,
    models_dir='models',
    input_txt='Hi'
):
    """
    Interactively run the model
    :model_name=124M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)
    """
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k, top_p=top_p
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        
        raw_text = input_txt
        context_tokens = enc.encode(raw_text)
        generated = 0
        
        out = sess.run(output, feed_dict={
            context: [context_tokens] })

        
        text = enc.decode(out[0])        
        print("=" * 40 + " SAMPLE " + " " + "=" * 40)
        print(text)
            
            

In [25]:
%%timeit
interact_model()

INFO:tensorflow:Restoring parameters from models/1558M/model.ckpt
======================================== SAMPLE  ========================================
Hi, everyone! It is time to recap this episode of Around the Verse. As with any follow up, Calix Reneau and Ryan McCaffrey will be your hosts for this one. Also, there will be no changes to any of your questions, just an easy blog update. Finally, Calix and I have another preview lined up for next week where we'll talk about what we've been doing since the last time we talked. We hope you have enjoyed the on-going updates. I'll see you all then.

[27:15] Calix Reneau (CDR): Welcome to Around the Verse, I'm Calix Reneau and you are listening to TSP. As I said during last week's top three, the second episode of Around the Verse was… well, really one episode entirely. Episode thirteen in all, one extremely excited fan writing his own episode. He's in the process of crafting it but it'll be filled with GOT loot and first time is never a

KeyboardInterrupt: 